<a href="https://colab.research.google.com/github/mahfouz72/softmax-and-neural-network-mnist/blob/master/softmax_and_neural_network_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("hi")

hi
